In this notebook we are going to use functionality provided by the SimMLA (pronounced 'Sim-L-A') package to compute the excitation irradiance profile in an epi-illumination fluorescence microscope using a fly's eye condenser. Such a condenser is realized with a pair of microlens arrays (MLA's). 

In [1]:
%pylab
import SimMLA.fftpack as simfft
import SimMLA.grids   as grids
plt.style.use('dark_background')
plt.rcParams['image.cmap'] = 'plasma'

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


# Setup the MLA geometry
We'll start by defining the geometry of the MLA's.
+ Prior to the objective there are two MLA's with the same parameters.
+ They are spaced by one focal length such that the second MLA is in the focal plane of the first.
+ Each lenslet has the same focal length as all the other lenslets.

In [2]:
numLenslets = 11    # Must be odd; corresponds to the number of lenslets in one dimension
lensletSize = 500   # microns
focalLength = 13700 # microns

wavelength  = 0.642 # microns

Now we'll establish a GridArray to represent the microlenses. Such an array is a coordinate system onto which the individual lenslets are mapped. The GridArray is primarily a convenience class because it manages all unit conversions when performing Fourier transforms.

The most important parameter here is the subgridSize. This is the number of discrete lattice sites into which a lenslet will be divided. The spatial sampling period for each lenslet will therefore be

$$\delta = \frac{p}{N_{sub}}$$

where $p$ is the MLA pitch in microns and $ N_{\text{sub}} $ is the subgrid size, i.e. the number of discrete lattice sites along one dimension of the MLA. For example, a MLA pitch of $p = 500 \, \mu m$ and a subgrid size of $N_{\text{sub}} = 51$ will result in a spatial sampling period of $\frac{500 \, \mu m}{51} \approx 9.8 \mu m$.

A good sampling rate should be smaller than the fastest oscillations in the amplitude and phase of the Fourier transform of the input field. This will probably mean that the best sampling rate is half the wavelength; unfortunately, this sampling rate will probably be too high for fast numerical computations, so we'll start with a more reasonable sampling rate.

In [3]:
subgridSize  = 501                       # Number of grid (or lattice) sites for a single lenslet
physicalSize = numLenslets * lensletSize # The full extent of the MLA

grid = grids.GridArray(numLenslets, subgridSize, physicalSize, wavelength, focalLength)

# Define the input field
Next, we'll define an input laser beam that will impinge upon the first MLA.

In [4]:
power   = 100  # mW
beamStd = 1000 # microns
uIn     = lambda x, y: np.sqrt(power) / (2 * np. pi * beamStd**2) * np.exp(-(x**2 + y**2) / 2 / beamStd**2)

plt.imshow(uIn(grid.px, grid.py),
           extent = (grid.px.min(), grid.px.max(), grid.py.min(), grid.py.max()))
plt.xlabel('x-position, microns')
plt.ylabel('y-position, microns')
plt.show()

# Find the field immediately after the second MLA
The field immediately *after* the second MLA is a summation of the Fourier transforms of all the fields sampled by each lenslet in the first array. Each transform is centered around the axis of its corresponding lenslet. Therefore, we have to compute a Fourier transform for each lenslet separately and then shift the origin of its coordinate axes onto the axis for the corresponding lenslet.

The reason that there is no quadratic phase curvature preceding the transforms is because the second MLA acts as a field lens, effectively canceling the quadratic phase terms.

To compute this parallelized Fourier transform, we use SimMLA's fftSubgrid routine. It will return two lists of interpolated fields, one for the magnitude and one for the phase. We will then resample this field onto a new grid representing the coordinate axes immediately after the second MLA.

*The reason we interpolate the resulting fields is because the physical units that the grid is built on change when the Fourier transform is performed. To return results based on the input grid spacing would require downsampling anyway, so I figured it would be better to let the user define the new grid spacing for their needs.*

In [5]:
# Compute the interpolated fields.
%time interpMag, interpPhase = simfft.fftSubgrid(uIn, grid)

CPU times: user 53min 45s, sys: 7min 37s, total: 1h 1min 22s
Wall time: 1h 5min 3s


Now that we have the interpolated fields, we'll define a new grid for sampling them. We'll give the grid the same physical extent as before, but we may choose to increase the spatial sampling rate slightly. We will also use the focal length of the objective because it will serve as the final Fourier transforming lens in the system.

*If we increase the sampling rate for the new grid, it won't necessarily fix any subsampling that occurred in the previous step. It will only provide better resolution at the possibly aliased results.*

In [6]:
%%time
fObj        = 3300 # microns
newGridSize = subgridSize * numLenslets # microns

newGrid = grids.Grid(newGridSize, physicalSize, wavelength, fObj)
field   = np.zeros((newGrid.gridSize, newGrid.gridSize))


# For each interpolated magnitude and phase corresponding to a lenslet
# 1) Compute the full complex field
# 2) Sum it with the other complex fields
for currMag, currPhase in zip(interpMag, interpPhase):
    fieldMag   = currMag(np.unique(newGrid.py), np.unique(newGrid.px))
    fieldPhase = currPhase(np.unique(newGrid.py), np.unique(newGrid.px))
    
    currField = fieldMag * np.exp(1j * fieldPhase)
    field = field + currField

CPU times: user 19min 55s, sys: 3min 19s, total: 23min 14s
Wall time: 28min 42s


In [15]:
fig, (ax0, ax1) = plt.subplots(nrows = 1, ncols = 2, sharey = True)
ax0.imshow(np.abs(field),
           interpolation = 'nearest',
           extent = (newGrid.px.min(), newGrid.px.max(), newGrid.py.min(), newGrid.py.max()))
ax0.set_xlabel('x-position, microns')
ax0.set_ylabel('y-position, microns')

ax1.imshow(np.angle(field),
           interpolation = 'nearest',
           extent = (newGrid.px.min(), newGrid.px.max(), newGrid.py.min(), newGrid.py.max()))
ax1.set_xlabel('x-position, microns')
plt.show()

In [7]:
np.save('field', field)

In [14]:
plt.imshow(np.abs(np.fft.fftshift(np.fft.fft2(field))),
           interpolation = 'nearest',
           extent = (newGrid.pX.min(), newGrid.pX.max(), newGrid.pY.min(), newGrid.pY.max()))